# Calibration Tutorial - Fork Peck, MT - Unirrigated Flux Plot

## Step 1: Getting the Data

Note: Step 1 can be bypassed if necessary. Just unzip `tutorials/2_Fort_Peck/data/prepped_input.zip` and proceed to Step 2.

This tutorial is going to focus on calibrating the SWIM for a single unirrigated plot, which is a 150 meter buffer around FluxNet's US-Fpe eddy covariance station. This will be interesting because the flux station has independent observations of both meteorology and the ET flux itself. Let's see if we can improve upon our *a priori* estimates of soil hydraulic parameters and our default parameterization of the snow and Kcb model (function of NDVI).

This notebook bases all it's work on the polygons in `tutorials/2_Fort_Peck/data/gis/flux_fields.shp`.

This notebook is easily modified to get all the flux station's input data; just remove the `select` or like arguments (`field_select`/`target_fields`, etc.) so it doesn't run on only our focus, the station at Fort Peck. This is what Tutorial 4 does. This would additionally require downloading the entire bias correction GeoTIFF data set, from the Earth Engine asset folder `projects/ee-dgketchum/assets/swim/gridmet_bias_correction`.

We won't cover the details of compiling data for the modeling run unless it is a different approach than what we did in the first tutorial, `1_Boulder`. Revisit that tutorial's steps 2-4 for further explanation of what we speed through below. Let's get started:

In [8]:
import os
import sys
import json

import geopandas as gpd

root = os.path.abspath('../../..')
sys.path.append(root)

from swimrs.data_extraction.ee.ee_utils import is_authorized
from swimrs.data_extraction.ee.etf_export import sparse_sample_etf
from swimrs.data_extraction.ee.ndvi_export import sparse_sample_ndvi
from swimrs.data_extraction.ee.snodas_export import sample_snodas_swe
from swimrs.data_extraction.snodas.snodas import create_timeseries_json
from swimrs.data_extraction.ee.ee_props import get_irrigation, get_ssurgo

from swimrs.utils.google_bucket import list_and_copy_gcs_bucket

from swimrs.data_extraction.gridmet.gridmet import assign_gridmet_and_corrections
from swimrs.data_extraction.gridmet.gridmet import download_gridmet

from swimrs.prep.field_properties import write_field_properties
from swimrs.prep.remote_sensing import sparse_time_series, join_remote_sensing
from swimrs.prep.dynamics import process_dynamics_batch

from swimrs.prep.field_timeseries import join_daily_timeseries
from swimrs.prep.prep_plots import prep_fields_json

from swimrs.swim.config import ProjectConfig

# Set matplotlib to display inline
%matplotlib inline

home = os.path.expanduser('~')
command = os.path.join(home, 'google-cloud-sdk/bin/gsutil')

# Export destination switch
drive = True  # default: export to Google Drive
# To use a Cloud Storage bucket instead (faster), uncomment and set your bucket:
drive = False
bucket = 'your-bucket-name'
bucket = None if 'bucket' not in globals() else bucket
export_dest = 'drive' if drive else 'bucket'
export_bucket = None if drive else bucket

We will use our configuration file to instantiate a `ProjectConfig` object that holds all configuration details, including paths. This will simplify our lives:

In [9]:
root = os.path.abspath('../../..')
project_ws = os.path.join(root, 'examples', '2_Fort_Peck')
data = os.path.join(project_ws, 'data')
config_path = os.path.join(project_ws, 'config.toml')

config = ProjectConfig()
config.read_config(config_path, project_root_override=project_ws)

In [10]:
gdf = gpd.read_file(config.fields_shapefile)
gdf.shape

AttributeError: 'ProjectConfig' object has no attribute 'shapefile'

In [ ]:
gdf.columns

The shapefile is derived from John Volk's work, which is documented here: https://www.sciencedirect.com/science/article/pii/S0168192323000011

Their paper used a sophisticated method for determining the area on the ground that contibutes to the flux observation. Instead of using their footprints, we're going to use a 150-m buffer around the station for simplicity. We're just going to use one station from the dataset, US-FPe.

In [ ]:
FEATURE_ID = config.feature_id_col
gdf.index = gdf[FEATURE_ID]
selected_feature = 'US-FPe'
gdf = gdf.loc[[selected_feature]].copy()
gdf

In [ ]:
centroid = gdf.geometry.centroid
x, y = centroid.x, centroid.y

In [ ]:
import contextily as ctx
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10, 10), subplot_kw={'projection': ccrs.epsg(5071)})

gdf.plot(ax=ax,
         edgecolor='black',
         facecolor='yellow',
         alpha=0.3,
         legend=True,
         transform=ccrs.epsg(5071))

gdf.geometry.centroid.plot(ax=ax, markersize=10, color='red', transform=ccrs.epsg(5071))

# Set the extent (x0, x1, y0, y1) to "zoom out"
ax.set_extent([x - 1000, x + 1000, y - 1000, y + 1000], crs=ccrs.epsg(5071))
ctx.add_basemap(ax, source=ctx.providers.Esri.WorldImagery, crs=ccrs.epsg(5071))

plt.title(selected_feature)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()


Instead of using the `clustered_sample_etf` like in the first tutorial, we're going to use `sparse_sample_etf` since we only have one station. We would use the same if we selected from more stations, as they are spread out. The`sparse_sample_etf` function accepts a shapefile.

We can use the `select` argument and use a list of our single feature ID to only export that data. 

Note: In this tutorial, you can modify the function calls to export data for all the flux stations easily; just remove the `select` arguments, and it will get all the data needed for the validation tutorial.

### 1. Earth Engine Remote Sensing Extracts

In [ ]:
landsat = os.path.join(config.data_dir, 'landsat')
extracts = os.path.join(landsat, 'extracts')
tables = os.path.join(landsat, 'tables')
if not os.path.isdir(tables):
    os.makedirs(tables, exist_ok=True)

In [31]:
# The extraction
is_authorized()
bucket = 'wudr'

for src in ['ndvi', 'etf']:
    for mask in ['inv_irr', 'irr']:

        dst = os.path.join(config.data_dir, 'landsat', src, mask)

        if not os.path.exists(dst):
            os.makedirs(dst, exist_ok=True)

        if src == 'etf':
            print(src, mask)
            sparse_sample_etf(shapefile_path, bucket=export_bucket, debug=False, mask_type=mask, check_dir=dst,
                              dest=export_dest, drive_folder='swim', drive_categorize=True, start_yr=1987, end_yr=2022,
                              feature_id=FEATURE_ID, select=None)
        if src == 'ndvi':
            print(src, mask)
            sparse_sample_ndvi(shapefile_path, bucket=export_bucket, debug=False, mask_type=mask, check_dir=dst,
                               dest=export_dest, drive_folder='swim', drive_categorize=True, start_yr=1987, end_yr=2022,
                               feature_id=FEATURE_ID, select=None)


Authorized
ndvi inv_irr
ndvi_US-A32_inv_irr_1987
ndvi_US-A32_inv_irr_1988
ndvi_US-A32_inv_irr_1989
ndvi_US-A32_inv_irr_1990
ndvi_US-A32_inv_irr_1991
ndvi_US-A32_inv_irr_1992
ndvi_US-A32_inv_irr_1993
ndvi_US-A32_inv_irr_1994
ndvi_US-A32_inv_irr_1995
ndvi_US-A32_inv_irr_1996
ndvi_US-A32_inv_irr_1997
ndvi_US-A32_inv_irr_1998
ndvi_US-A32_inv_irr_1999
ndvi_US-A32_inv_irr_2000
ndvi_US-A32_inv_irr_2001
ndvi_US-A32_inv_irr_2002
ndvi_US-A32_inv_irr_2003
ndvi_US-A32_inv_irr_2004
ndvi_US-A32_inv_irr_2005
ndvi_US-A32_inv_irr_2006
ndvi_US-A32_inv_irr_2007
ndvi_US-A32_inv_irr_2008
ndvi_US-A32_inv_irr_2009
ndvi_US-A32_inv_irr_2010
ndvi_US-A32_inv_irr_2011
ndvi_US-A32_inv_irr_2012
ndvi_US-A32_inv_irr_2013
ndvi_US-A32_inv_irr_2014
ndvi_US-A32_inv_irr_2015
ndvi_US-A32_inv_irr_2016
ndvi_US-A32_inv_irr_2017
ndvi_US-A32_inv_irr_2018
ndvi_US-A32_inv_irr_2019
ndvi_US-A32_inv_irr_2020
ndvi_US-A32_inv_irr_2021
ndvi_US-A32_inv_irr_2022
ndvi_US-A74_inv_irr_1987
ndvi_US-A74_inv_irr_1988
ndvi_US-A74_inv_irr_1989
n

EEException: Too many tasks already in the queue (3000, limit 3000).

In [33]:
# Copy to local directory; Note the flux-station specific for-loop

# use the following for a full station network extract:
stations = gdf[FEATURE_ID].tolist()

# # use this for just Fort Peck
# stations = [selected_feature]

for src in ['ndvi', 'etf']:
    for mask in ['inv_irr', 'irr']:
        for station in stations:
            dst = os.path.join(config.data_dir, 'landsat', 'extracts', src, mask)
            if not os.path.isdir(dst):
                os.makedirs(dst, exist_ok=True)
            glob_ = f'{src}_{station}_{mask}'
            if not drive and export_bucket:
                list_and_copy_gcs_bucket(command, export_bucket, dst, glob=glob_, dry_run=False, overwrite=False)
            else:
                print('Drive exports: download the CSVs from Drive into', dst)

Copying gs://wudr/ndvi_US-A32_inv_irr_1987.csv: Copying gs://wudr/ndvi_US-A32_inv_irr_1987.csv...
/ [1 files][  932.0 B/  932.0 B]                                                
Operation completed over 1 objects/932.0 B.                                      

Copied gs://wudr/ndvi_US-A32_inv_irr_1987.csv to /home/dgketchum/PycharmProjects/swim-rs/tutorials/2_Fort_Peck/data/landsat/extracts/ndvi/inv_irr
Copying gs://wudr/ndvi_US-A32_inv_irr_1988.csv: Copying gs://wudr/ndvi_US-A32_inv_irr_1988.csv...
/ [1 files][  910.0 B/  910.0 B]                                                
Operation completed over 1 objects/910.0 B.                                      

Copied gs://wudr/ndvi_US-A32_inv_irr_1988.csv to /home/dgketchum/PycharmProjects/swim-rs/tutorials/2_Fort_Peck/data/landsat/extracts/ndvi/inv_irr
Copying gs://wudr/ndvi_US-A32_inv_irr_1989.csv: Copying gs://wudr/ndvi_US-A32_inv_irr_1989.csv...
/ [1 files][  844.0 B/  844.0 B]                                                
Opera

KeyboardInterrupt: 

In [2]:
# Write to interpolated time series

types_ = ['inv_irr', 'irr']
sensing_params = ['ndvi', 'etf']

for mask_type in types_:

    for sensing_param in sensing_params:
        yrs = [x for x in range(1987, 2024)]

        ee_data, src = None, None

        ee_data = os.path.join(extracts, sensing_param, mask_type)
        src = os.path.join(tables, '{}_{}_{}.csv'.format('tutorial', sensing_param, mask_type))
        src_ct = os.path.join(tables, '{}_{}_{}_ct.csv'.format('tutorial', sensing_param, mask_type))

        sparse_landsat_time_series(shapefile_path, ee_data, yrs, src, src_ct, feature_id=FEATURE_ID,
                                   select=[selected_feature])

/home/dgketchum/PycharmProjects/swim-rs/tutorials/2_Fort_Peck/data/landsat/extracts/ndvi/inv_irr
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
/home/dgketchum/PycharmProjects/swim-rs/tutorials/2_Fort_Peck/data/landsat/extracts/etf/inv_irr
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
/home/dgketchum/PycharmProjects/swim-rs/tutorials/2_Fort_Peck/data/landsat/extracts/ndvi/irr
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
/home/dgketchum/PycharmProjects/swim-rs/tutorials/2_Fort_Peck/data/landsat/extracts/etf/irr
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
20

In [12]:
remote_sensing_file = os.path.join(landsat, 'remote_sensing.csv')
join_remote_sensing(tables, remote_sensing_file)

etf_inv_irr_ct
ndvi_inv_irr_ct
etf_inv_irr
etf_irr
etf_irr_ct
ndvi_inv_irr
ndvi_irr_ct
ndvi_irr


### 2. Other Earth Engine Extracts

In [22]:
snow_dst = os.path.join(config.data_dir, 'snodas', 'extracts')
if not os.path.isdir(snow_dst):
    os.makedirs(snow_dst, exist_ok=True)

In [ ]:
# Use local shapefile (no EE asset upload required)
shapefile_path = config.fields_shapefile
sample_snodas_swe(shapefile_path, bucket=export_bucket, debug=False, check_dir=snow_dst, overwrite=False,
                  feature_id=FEATURE_ID, dest=export_dest, drive_folder='swim', drive_categorize=True)

description = 'calibration_irr'
get_irrigation(shapefile_path, description, debug=False, selector=FEATURE_ID, lanid=True, dest=export_dest,
               drive_folder='swim', drive_categorize=True)

description = 'calibration_ssurgo'
get_ssurgo(shapefile_path, description, debug=False, selector=FEATURE_ID, dest=export_dest, drive_folder='swim',
           drive_categorize=True)

In [ ]:
glob_ = 'swe'
if not drive and export_bucket:
    list_and_copy_gcs_bucket(command, export_bucket, snow_dst, glob=glob_, dry_run=False, overwrite=False)
else:
    print('Drive exports: download SNODAS CSVs from Drive into', snow_dst)
snow = os.path.join(config.data_dir, 'snodas', 'snodas.json')
create_timeseries_json(snow_dst, snow, feature_id=FEATURE_ID)

In [ ]:
description = 'calibration_ssurgo'
dst = os.path.join(config.data_dir, 'properties')
if not drive and export_bucket:
    list_and_copy_gcs_bucket(command, export_bucket, dst, glob=description, dry_run=False, overwrite=False)
else:
    print('Drive exports: download irrigation CSV(s) from Drive into', dst)

In [7]:
description = 'calibration_irr'
dst = os.path.join(data, 'properties')
if not drive and export_bucket:
    list_and_copy_gcs_bucket(command, export_bucket, dst, glob=description, dry_run=False, overwrite=False)
else:
    print('Drive exports: download SSURGO CSV(s) from Drive into', dst)

/home/dgketchum/PycharmProjects/swim-rs/tutorials/2_Fort_Peck/data/properties/calibration_irr.csv exists, skipping


In [13]:
irr = os.path.join(config.data_dir, 'properties', 'calibration_irr.csv')
ssurgo = os.path.join(config.data_dir, 'properties', 'calibration_ssurgo.csv')
properties_json = os.path.join(config.data_dir, 'properties', 'calibration_properties.json')

write_field_properties(shp=config.fields_shapefile, irr=irr, soils=ssurgo, out_js=properties_json, index_col=FEATURE_ID,
                       shp_add=None,
                       targets=[selected_feature])

We can see the Fort Peck location has never been irrigated, according to IrrMapper.

In [14]:
with open(properties_json, 'r') as f:
    properties_dict = json.load(f)
properties_dict[selected_feature]

{'irr': {'1987': 0.0,
  '1988': 0.0,
  '1989': 0.0,
  '1990': 0.0,
  '1991': 0.0,
  '1992': 0.0,
  '1993': 0.0,
  '1994': 0.0,
  '1995': 0.0,
  '1996': 0.0,
  '1997': 0.0,
  '1998': 0.0,
  '1999': 0.0,
  '2000': 0.0,
  '2001': 0.0,
  '2002': 0.0,
  '2003': 0.0,
  '2004': 0.0,
  '2005': 0.0,
  '2006': 0.0,
  '2007': 0.0,
  '2008': 0.0,
  '2009': 0.0,
  '2010': 0.0,
  '2011': 0.0,
  '2012': 0.0,
  '2013': 0.0,
  '2014': 0.0,
  '2015': 0.0,
  '2016': 0.0,
  '2017': 0.0,
  '2018': 0.0,
  '2019': 0.0,
  '2020': 0.0,
  '2021': 0.0,
  '2022': 0.0,
  '2023': 0.0,
  '2024': 0.0},
 'awc': 0.1775607833906229,
 'ksat': 8.683497246126228,
 'clay': 26.05900683846139,
 'sand': 13.162341057404172,
 'area_sq_m': 69528.82373434979}

While the Fort Peck site isn't irrigated, we will run the NDVI analysis anyway so we don't have to write dummy data required for the model run. 

Note: There is an approach to force the model to run on unirrigated lands that we will see later.

In [15]:
cuttings_json = os.path.join(landsat, 'calibration_cuttings.json')
# Update: run dynamics after joining the daily time series.
# process_dynamics_batch(joined_timeseries, properties_json, cuttings_json, etf_target='ssebop', irr_threshold=0.1,
#                        select=[selected_feature], masks=('irr','inv_irr'), instruments=('landsat',),
#                        use_lulc=False, use_mask=True, lookback=5, num_workers=1)

Analyzing Irrigation: 100%|███████████████| 161/161 [00:00<00:00, 146355.21it/s]

wrote /home/dgketchum/PycharmProjects/swim-rs/tutorials/2_Fort_Peck/data/landsat/calibration_cuttings.json


### 3. Meteorology Data

Since we are analyzing scattered stations, we accept we will need a different GridMET time series for each station, so we won't save any time associating each field with indexed GridMET cells (like we did with the cluster of fields). Therefore instead of using `find_gridmet_points` as we did in the previous tutorial, we can use the faster `get_gridmet_corrections`, which outputs the same things: a shapefile with the `GFID`s and a `json` file that has each GFID monthly correction factor. The difference is that the `GFID` is just a sequence of integers. This is all we need to proceed with downloading and process the GridMET data.

This has the added benefit of removing the dependency on a vector representation of the GridMET grid, which requires downloading a raster, vectorizing it, and manipulating that shapefile, which is described in the docstring of `find_gridmet_points`.

If running extractions for the entire flux network, you will need to replace the bias correction GeoTIFF residing in this tutorial's `data` directory with the CONUS rasters from Earth Engine (`projects/ee-dgketchum/assets/swim/gridmet_bias_correction`). See https://developers.google.com/earth-engine/guides/exporting_images for tips on downloading.

Let's run the function to extract correction factors and download the data for Fort Peck:

In [ ]:
correction_tifs = os.path.join(config.data_dir, 'bias_correction_tif')

fields_gridmet = os.path.join(config.data_dir, 'gis', 'flux_fields_gfid.shp')

gridmet_factors = os.path.join(config.data_dir, 'gis', 'flux_fields_gfid.json')

met = os.path.join(config.data_dir, 'met_timeseries')
if not os.path.isdir(met):
    os.mkdir(met)

In [ ]:
assign_gridmet_and_corrections(fields=shapefile_path,
                               gridmet_ras=correction_tifs,
                               fields_join=fields_gridmet,
                               factors_js=gridmet_factors,
                               feature_id=FEATURE_ID,
                               field_select=None)

In [ ]:
download_gridmet(fields_gridmet, gridmet_factors, met, start='1987-01-01', end='2023-12-31',
                 target_fields=[selected_feature], overwrite=False, feature_id=None)

### 4. Model Preparation

Now that we have out data together, the rest of the prep process is the same as in the first tutorial.

In [25]:
joined_timeseries = os.path.join(data, 'plot_timeseries')
if not os.path.isdir(joined_timeseries):
    os.mkdir(joined_timeseries)

In [26]:
params = ['etf_inv_irr',
          'ndvi_inv_irr',
          'etf_irr',
          'ndvi_irr']
params += ['{}_ct'.format(p) for p in params]

join_daily_timeseries(fields=fields_gridmet, met_dir=met, rs_dir=remote_sensing_file,
                      dst_dir=joined_timeseries, snow=snow, overwrite=True, start_date='1987-01-01',
                      end_date='2022-12-31', feature_id=FEATURE_ID, **{'params': params})

100%|█████████████████████████████████████████████| 1/1 [00:02<00:00,  2.25s/it]

1 fields were successfully processed
0 fields were dropped due to missing data


In [30]:
# the model input file
processed_targets, excluded_targets = prep_fields_json(properties_json, joined_timeseries, config.input_data,
                                                       dynamics=cuttings_json, target_plots=None)

Target sample(s) missing: ['US-Tw2', 'US-Slt', 'US-Fuf', 'US-xNG', 'S2', 'US-Hn3', 'US-xDL', 'US-Wkg', 'WRV_1', 'US-SCs', 'UA2_KN20', 'KV_4', 'WRV_2', 'US-SO4', 'US-A32', 'B_01', 'US-Mj2', 'US-Ced', 'US-SP4', 'US-xNW', 'US-Me2', 'BPHV', 'LYS_SE', 'US-Sne', 'US-Aud', 'US-Hn2', 'US-xJR', 'RIP760', 'US-Me6', 'US-SCg', 'US-Ro4', 'JPL1_JV114', 'SPV_1', 'US-Oho', 'stonevillesoy', 'US-xDC', 'US-Skr', 'US-Blk', 'US-Ro3', 'UA3_JV108', 'BPLV', 'ET_1', 'US-SdH', 'US-Br1', 'US-SCw', 'US-ARM', 'UA1_KN18', 'US-NR1', 'US-Bi1', 'US-Bi2', 'US-Me1', 'US-SRG', 'UMVW', 'US-xSB', 'US-Ro5', 'US-Dk2', 'SV_5', 'BAR012', 'US-Ne1', 'US-Ro1', 'Ellendale', 'UA1_JV187', 'manilacotton', 'US-xYE', 'UA3_KN15', 'SPV_3', 'LYS_NE', 'US-WBW', 'US-Srr', 'US-ADR', 'US-MOz', 'US-SRS', 'US-NC2', 'US-GMF', 'US-Fwf', 'US-FR2', 'US-IB2', 'VR', 'US-CRT', 'ALARC2_Smith6', 'US-Dix', 'US-xDS', 'LYS_NW', 'US-Ne2', 'KV_1', 'US-WCr', 'US-xAE', 'UA1_HartFarm', 'Almond_Med', 'B_11', 'US-Fmf', 'SV_6', 'US-Twt', 'US-xRM', 'US-Ro2', 'US-Mj

100%|████████████████████████████████████████| 161/161 [00:00<00:00, 271.28it/s]


wrote /home/dgketchum/PycharmProjects/swim-rs/tutorials/2_Fort_Peck/data/prepped_input.json


Note: If you're just running the single station, the code will warn you're missing the other 160 stations. It's not a problem.

Hopefully, that went quick. In Step 2, we'll run an uncalibrated model.

### Retrieve ETf and NDVI from Google Drive

After exports finish in the Earth Engine task manager:
1) Open Google Drive. If `drive_categorize=True`:
   - ETf files under `swim_etf/` (e.g., `etf_irr_2004.csv`, `etf_inv_irr_2004.csv`)
   - NDVI files under `swim_ndvi/` (e.g., `ndvi_irr_2004.csv`, `ndvi_inv_irr_2004.csv`)
   Otherwise look under the `swim/` folder.
2) Create local extract folders:
   - `examples/2_Fort_Peck/data/landsat/extracts/etf/irr/` and `.../etf/inv_irr/`
   - `examples/2_Fort_Peck/data/landsat/extracts/ndvi/irr/` and `.../ndvi/inv_irr/`
3) Download Drive CSVs into those folders.
4) Verify the years in your study window are present for both masks before continuing.

Tip: If you have a Cloud Storage bucket and set `drive=False`:
- You can move files with `gsutil -m cp` or the helper `list_and_copy_gcs_bucket`.
- Bucket transfers are significantly faster for large projects.